# Restaurant quality and profile at the top 50 airports in the world

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
<a href="#item0">Context</a>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Top 50 Airports by Passenger Traffic</a>

3. <a href="#item3">Analyze the Restaurants at each airport and cluster the airports</a>

 3.1. <a href="#item31">Restaurant Quality</a>
    
 3.2. <a href="#item31">Restaurant Profile</a>

4. <a href="#item4">Examine Clusters</a>
    
 4.1. <a href="#item31">Restaurant Quality</a>
    
 4.2. <a href="#item31">Restaurant Profile</a>
    
5. <a href="#item5">Report</a>

 5.1. <a href="#item31">Introduction</a>
    
 5.2. <a href="#item31">Methodology</a>

 5.3. <a href="#item31">Results</a>

 5.4. <a href="#item31">Discussion</a>
    
 5.5. <a href="#item31">Conclusion</a>

</font>
</div>

<font size = 5><a href="#item0">Context</a> <br></font>
While flying becomes a commodity and passenger traffic rises ever more people spent time at and around big airports. Apart from internal transport infrastruce, connecting public transport, new amenities such as sleep pods, sports etc stil a great opportunity to differentiate are great restaurants, hotels and shopping opportunites. Airport operators need a means to compare themselves with other top airports across the world.
The following analysis will provide a starting point by looking at the 50 airports with the largest passenger turnover per year and compare them based on how their restaurants, hotels and shops where rated by passengers. The rating data is supplied by foursquare, the airport location data is supplied by https://openflights.org/data.html and the data for the passenger traffic comes from Wikipedia (https://en.wikipedia.org/wiki/List_of_busiest_airports_by_passenger_traffic). 



In [943]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import xlrd

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

#### Source: https://openflights.org/data.html

In [944]:
airport_data = pd.read_csv("airports.csv")
airport_data.head()

,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10.0,U,Pacific/Port_Moresby
1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.788700,20,10.0,U,Pacific/Port_Moresby
2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10.0,U,Pacific/Port_Moresby
3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10.0,U,Pacific/Port_Moresby
4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.220050,146,10.0,U,Pacific/Port_Moresby


#### Source: Wikipedia

In [945]:
top50_airports = pd.read_excel("airports_2016.xlsx")
top50_airports.head()

,rank,name,code,location,country,passengers
0,1,HartsfieldñJackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,"104,171,935"
1,2,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,"94,393,454"
2,3,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,"83,654,250"
3,4,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,"80,921,527"
4,5,Tokyo International Airport,HND/RJTT,"Ota, Tokyo",Japan,"79,699,762"


In [946]:
def split_code(row):
    code = row.split("/")[0]
    icao = row.split("/")[1]
    return code

In [947]:
top50_airports["code_new"] = top50_airports["code"].apply(split_code)

In [948]:
top50_airports.head()

,rank,name,code,location,country,passengers,code_new
0,1,HartsfieldñJackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,"104,171,935",ATL
1,2,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,"94,393,454",PEK
2,3,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,"83,654,250",DXB
3,4,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,"80,921,527",LAX
4,5,Tokyo International Airport,HND/RJTT,"Ota, Tokyo",Japan,"79,699,762",HND


In [949]:
top50_airports.drop("code", axis=1, inplace=True)

In [950]:
top50_airports.rename({"code_new":"code"}, axis=1, inplace=True)

In [951]:
my_data = top50_airports.merge(airport_data, how="left", on="code")

In [952]:
my_data.head()

,rank,name_x,location,country_x,passengers,code,id,name_y,city,country_y,icao,latitude,longitude,altitude,offset,dst,timezone
0,1,HartsfieldñJackson Atlanta International Airport,"Atlanta, Georgia",United States,"104,171,935",ATL,3682,Hartsfield Jackson Atlanta Intl,Atlanta,United States,KATL,33.636719,-84.428067,1026,-5.0,A,America/New_York
1,2,Beijing Capital International Airport,"Chaoyang-Shunyi, Beijing",China,"94,393,454",PEK,3364,Capital Intl,Beijing,China,ZBAA,40.080111,116.584556,116,8.0,U,Asia/Chongqing
2,3,Dubai International Airport,"Garhoud, Dubai",United Arab Emirates,"83,654,250",DXB,2188,Dubai Intl,Dubai,United Arab Emirates,OMDB,25.252778,55.364444,62,4.0,U,Asia/Dubai
3,4,Los Angeles International Airport,"Los Angeles, California",United States,"80,921,527",LAX,3484,Los Angeles Intl,Los Angeles,United States,KLAX,33.942536,-118.408075,126,-8.0,A,America/Los_Angeles
4,5,Tokyo International Airport,"Ota, Tokyo",Japan,"79,699,762",HND,2359,Tokyo Intl,Tokyo,Japan,RJTT,35.552258,139.779694,35,9.0,U,Asia/Tokyo


In [953]:
my_data.columns

Index(['rank', 'name_x', 'location', 'country_x', 'passengers', 'code', 'id',
       'name_y', 'city', 'country_y', 'icao', 'latitude', 'longitude',
       'altitude', 'offset', 'dst', 'timezone'],
      dtype='object')

In [954]:
my_data.drop(['rank', 'passengers', 'code',
       'name_y','country_y', 'icao',"id",
       'altitude', 'offset', 'dst', 'timezone'], axis=1, inplace=True)

In [955]:
my_data.columns = ["name", "location", "country", "city", "latitude", "longitude"]

## 2. Explore Top 50 Airports by Passenger Traffic

In [956]:
my_data.head()

,name,location,country,city,latitude,longitude
0,HartsfieldñJackson Atlanta International Airport,"Atlanta, Georgia",United States,Atlanta,33.636719,-84.428067
1,Beijing Capital International Airport,"Chaoyang-Shunyi, Beijing",China,Beijing,40.080111,116.584556
2,Dubai International Airport,"Garhoud, Dubai",United Arab Emirates,Dubai,25.252778,55.364444
3,Los Angeles International Airport,"Los Angeles, California",United States,Los Angeles,33.942536,-118.408075
4,Tokyo International Airport,"Ota, Tokyo",Japan,Tokyo,35.552258,139.779694


In [957]:
# create map of Manhattan using latitude and longitude values
map_top50 = folium.Map(location=[51.509865, -0.118092], zoom_start=2)

# add markers to map
for lat, lng, label in zip(my_data['latitude'], my_data['longitude'], my_data['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,).add_to(map_top50) 

folium.Marker(
    [85, -45],
    icon=DivIcon(
        icon_size=(600,200),
        icon_anchor=(100,0),
        html='<div style="font-size: 18pt">Top 50 Airports by passenger traffic</div>')
    ).add_to(map_top50)
    
map_top50

## 3. Analyze the Restaurants at each airport and cluster the airports

### 3.1. Restaurant Quality

In [1155]:
CLIENT_ID = 'BMMZILCYQCZJE1KR2SJYMHMFLDBQBMRZOXFRN3C3AARRX113'
CLIENT_SECRET= 'C1GY15IRDEV4W0KO2U1FZXMLSE2PH33O50FB1FEVMW0G4BPI'
VERSION = '20180604'
LIMIT = 100
radius = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BMMZILCYQCZJE1KR2SJYMHMFLDBQBMRZOXFRN3C3AARRX113
CLIENT_SECRET:C1GY15IRDEV4W0KO2U1FZXMLSE2PH33O50FB1FEVMW0G4BPI


#### test for one airport

In [1156]:
airport_latitude = my_data.loc[12, 'latitude'] # aiport latitude value
airport_longitude = my_data.loc[12, 'longitude'] # airport longitude value

airport_name = my_data.loc[12, 'name'] # airport name

print('Latitude and longitude values of {} and {}, {}.'.format(airport_latitude, airport_longitude, airport_name))

Latitude and longitude values of 50.026421 and 8.543125, Frankfurt Airport.


In [1157]:
temp_lim = 30
radius=3000
search_term="restaurant"
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    airport_latitude, 
    airport_longitude, 
    radius, 
    temp_lim,
    search_term)
url

'https://api.foursquare.com/v2/venues/search?&client_id=BMMZILCYQCZJE1KR2SJYMHMFLDBQBMRZOXFRN3C3AARRX113&client_secret=C1GY15IRDEV4W0KO2U1FZXMLSE2PH33O50FB1FEVMW0G4BPI&v=20180604&ll=50.026421,8.543125&radius=3000&limit=30&query=restaurant'

In [1158]:
results = requests.get(url).json()

In [1159]:
# assign relevant part of JSON to venues
venues = results['response']['venues']


# tranform venues into a dataframe
dataframe = json_normalize(venues)
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered.head()

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,id
0,Restaurant Taverne,"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",Hugo-Eckener-Ring 15,DE,Frankfurt am Main,Deutschland,FRA Airport,3525,"[Hugo-Eckener-Ring 15 (FRA Airport), 60549 Fra...","[{'label': 'display', 'lat': 50.05194537947098...",50.051945,8.572313,60549,Hessen,4debe04952b11677f06109ad
1,Around the World Restaurant,"[{'id': '4bf58dd8d48988d1ef931735', 'name': 'A...",Terminal 1,DE,Frankfurt am Main,Deutschland,FRA Airport,3274,"[Terminal 1 (FRA Airport), 60549 Frankfurt am ...","[{'label': 'display', 'lat': 50.05025430486973...",50.050254,8.569973,60549,Hessen,4d08c1e31657a35d564d2ee7
2,Restaurant Zagreb,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",NaN,DE,Mörfelden-Walldorf,Deutschland,NaN,2861,"[Mörfelden-Walldorf, Deutschland]","[{'label': 'display', 'lat': 50.00608826631427...",50.006088,8.567610,NaN,Hessen,4df9ce6d18a8ab1870bc1823
3,MONDO Bar & Restaurant,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","Flughafen Frankfurt, FRA, Terminal 2",DE,Frankfurt am Main,Deutschland,NaN,2631,"[Flughafen Frankfurt, FRA, Terminal 2, 60549 F...","[{'label': 'display', 'lat': 50.03820895358539...",50.038209,8.575029,60549,Hessen,58da577b51666a1257891328
4,nhube restaurant,"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",NaN,DE,Frankfurt am Main,Deutschland,NaN,2648,"[Frankfurt am Main, Deutschland]","[{'label': 'display', 'lat': 50.050168, 'lng':...",50.050168,8.545495,NaN,Hessen,5a62488c0457b7595072055d


In [1160]:
len(results["response"]['venues'])

10

In [1161]:
results["response"]['venues'][1]

{'id': '4d08c1e31657a35d564d2ee7',
 'name': 'Around the World Restaurant',
 'location': {'address': 'Terminal 1',
  'crossStreet': 'FRA Airport',
  'lat': 50.05025430486973,
  'lng': 8.569972834823629,
  'labeledLatLngs': [{'label': 'display',
    'lat': 50.05025430486973,
    'lng': 8.569972834823629}],
  'distance': 3274,
  'postalCode': '60549',
  'cc': 'DE',
  'city': 'Frankfurt am Main',
  'state': 'Hessen',
  'country': 'Deutschland',
  'formattedAddress': ['Terminal 1 (FRA Airport)',
   '60549 Frankfurt am Main',
   'Deutschland']},
 'categories': [{'id': '4bf58dd8d48988d1ef931735',
   'name': 'Airport Food Court',
   'pluralName': 'Airport Food Courts',
   'shortName': 'Food Court',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1546785628',
 'hasPerk': False}

In [1162]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Restaurant Taverne,German Restaurant,Hugo-Eckener-Ring 15,DE,Frankfurt am Main,Deutschland,FRA Airport,3525,"[Hugo-Eckener-Ring 15 (FRA Airport), 60549 Fra...","[{'label': 'display', 'lat': 50.05194537947098...",50.051945,8.572313,60549,Hessen,4debe04952b11677f06109ad
1,Around the World Restaurant,Airport Food Court,Terminal 1,DE,Frankfurt am Main,Deutschland,FRA Airport,3274,"[Terminal 1 (FRA Airport), 60549 Frankfurt am ...","[{'label': 'display', 'lat': 50.05025430486973...",50.050254,8.569973,60549,Hessen,4d08c1e31657a35d564d2ee7
2,Restaurant Zagreb,Restaurant,NaN,DE,Mörfelden-Walldorf,Deutschland,NaN,2861,"[Mörfelden-Walldorf, Deutschland]","[{'label': 'display', 'lat': 50.00608826631427...",50.006088,8.567610,NaN,Hessen,4df9ce6d18a8ab1870bc1823
3,MONDO Bar & Restaurant,Italian Restaurant,"Flughafen Frankfurt, FRA, Terminal 2",DE,Frankfurt am Main,Deutschland,NaN,2631,"[Flughafen Frankfurt, FRA, Terminal 2, 60549 F...","[{'label': 'display', 'lat': 50.03820895358539...",50.038209,8.575029,60549,Hessen,58da577b51666a1257891328
4,nhube restaurant,German Restaurant,NaN,DE,Frankfurt am Main,Deutschland,NaN,2648,"[Frankfurt am Main, Deutschland]","[{'label': 'display', 'lat': 50.050168, 'lng':...",50.050168,8.545495,NaN,Hessen,5a62488c0457b7595072055d
5,Thong Thai,Thai Restaurant,"Terminal 1, Airport City Mall",DE,Frankfurt am Main,Deutschland,FRA Airport,3346,"[Terminal 1, Airport City Mall (FRA Airport), ...","[{'label': 'display', 'lat': 50.05098199142028...",50.050982,8.570117,60547,Hessen,4bdc7b622a3a0f47160bb3b6
6,New York Restaurant,Breakfast Spot,Am Weiher 20,DE,Kelsterbach,Deutschland,NaN,3177,"[Am Weiher 20, 65451 Kelsterbach, Deutschland]","[{'label': 'display', 'lat': 50.05266611488105...",50.052666,8.525649,65451,Hessen,55fad38a498e96fd1d1e04f6
7,Sheri Inn,Corporate Cafeteria,Terminal 1,DE,Frankfurt am Main,Deutschland,FRA Airport,3532,"[Terminal 1 (FRA Airport), 60549 Frankfurt am ...","[{'label': 'display', 'lat': 50.05177948572213...",50.051779,8.572834,60549,Hessen,50239896e4b0e4c99e5324ad
8,Airportclub,Restaurant,NaN,DE,NaN,Deutschland,NaN,3347,[Deutschland],"[{'label': 'display', 'lat': 50.0514260313368,...",50.051426,8.569132,NaN,NaN,4f72ccb6e4b08f62bdf667ad
9,Avocado,Mediterranean Restaurant,Nordendstr. 12,DE,Mörfelden,Deutschland,NaN,3030,"[Nordendstr. 12, 64564 Mörfelden, Deutschland]","[{'label': 'display', 'lat': 50.01420066615167...",50.014201,8.580985,64564,Hessen,4f57a73fe4b0f52af6d029d7


In [1163]:
venue_id = dataframe_filtered.loc[0, 'id'] # ID of the first venue
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

In [1164]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.8


## repeat for all airports

In [1165]:
def getNearbyVenues(names, latitudes, longitudes, radius=12000, LIMIT_to=20, query_term="restaurant"):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT_to,
            query_term)
   
        # make the GET request
        my_results = requests.get(url).json()["response"]["venues"]
        #print(my_results[0])
        # return only relevant information for each nearby venue
        try:
            for v in my_results:
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['id'],
                    v['name'], 
                    v['location']['lat'], 
                    v['location']['lng'],  
                    v['categories'][0]['name'])])
        except:
            pass

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['airport_name', 
                  'latitude', 
                  'longitude',
                  'Venue ID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']            
    
    return(nearby_venues)

In [1166]:
airport_restaurants = getNearbyVenues(names=my_data['name'],
                                   latitudes=my_data['latitude'],
                                   longitudes=my_data['longitude']
                                  )
airport_restaurants.head()

HartsfieldñJackson Atlanta International Airport
Beijing Capital International Airport
Dubai International Airport
Los Angeles International Airport
Tokyo International Airport
O'Hare International Airport
Heathrow Airport
Hong Kong International Airport
Shanghai Pudong International Airport
Charles de Gaulle Airport
Dallas/Fort Worth International Airport
Amsterdam Airport Schiphol
Frankfurt Airport
Istanbul Atat¸rk Airport
Guangzhou Baiyun International Airport
John F. Kennedy International Airport
Singapore Changi Airport
Denver International Airport
Seoul Incheon International Airport
Suvarnabhumi Airport
Indira Gandhi International Airport
SoekarnoñHatta International Airport
San Francisco International Airport
Kuala Lumpur International Airport
Madrid Barajas Airport
McCarran International Airport
Chengdu Shuangliu International Airport
Seattle-Tacoma International Airport
Chhatrapati Shivaji International Airport
Miami International Airport
Charlotte Douglas International Airpor

,airport_name,latitude,longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,HartsfieldñJackson Atlanta International Airport,33.636719,-84.428067,4ef3d2bbe300e92b9f504d7d,One Flew South Restaurant & Sushi Bar,33.640935,-84.425910,Sushi Restaurant
1,HartsfieldñJackson Atlanta International Airport,33.636719,-84.428067,5580a668498e2d6545e18762,Restaurant 356,33.651771,-84.398666,Restaurant
2,HartsfieldñJackson Atlanta International Airport,33.636719,-84.428067,4a85e8d0f964a52099ff1fe3,La Fiesta Mexican Restaurant,33.659636,-84.435043,Mexican Restaurant
3,HartsfieldñJackson Atlanta International Airport,33.636719,-84.428067,5060be37e4b095191be9188f,Paschal's Express,33.640769,-84.436247,Southern / Soul Food Restaurant
4,Beijing Capital International Airport,40.080111,116.584556,54c1cdee498e10a990c16ca7,Tai Hing Restaurant 太興茶餐廳,40.080972,116.588093,Cantonese Restaurant


In [1167]:
def get_venue_rating(curr_venue_id):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(curr_venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        return result['response']['venue']['rating']
    except:
        return None

In [1168]:
airport_restaurants["rating"] = airport_restaurants["Venue ID"].apply(get_venue_rating)

In [1169]:
airport_restaurants_stripped = airport_restaurants.drop(["latitude","longitude","Venue ID", "Venue","Venue Latitude","Venue Longitude"], axis=1)

In [1170]:
airport_restaurants_stripped = airport_restaurants_stripped.dropna()

In [1171]:
airport_restaurants_stripped.head()

,airport_name,Venue Category,rating
0,HartsfieldñJackson Atlanta International Airport,Sushi Restaurant,9.3
1,HartsfieldñJackson Atlanta International Airport,Restaurant,7.5
2,HartsfieldñJackson Atlanta International Airport,Mexican Restaurant,6.9
3,HartsfieldñJackson Atlanta International Airport,Southern / Soul Food Restaurant,7.3
4,Beijing Capital International Airport,Cantonese Restaurant,5.7


In [1172]:
len(set(airport_restaurants_stripped["airport_name"]))

33

In [1173]:
set(airport_restaurants_stripped["Venue Category"])

{'Airport Food Court',
 'American Restaurant',
 'Asian Restaurant',
 'Bar',
 'Beer Bar',
 'Beer Garden',
 'Breakfast Spot',
 'Brewery',
 'Buffet',
 'Café',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Cha Chaan Teng',
 'Chinese Restaurant',
 'Coffee Shop',
 'Cuban Restaurant',
 'Dessert Shop',
 'Dim Sum Restaurant',
 'Diner',
 'Dutch Restaurant',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'Food',
 'Food & Drink Shop',
 'Food Court',
 'French Restaurant',
 'Furniture / Home Store',
 'Gastropub',
 'German Restaurant',
 'Greek Restaurant',
 'Halal Restaurant',
 'Hawaiian Restaurant',
 'Hong Kong Restaurant',
 'Hotel Bar',
 'Indian Restaurant',
 'Indonesian Restaurant',
 'Italian Restaurant',
 'Japanese Curry Restaurant',
 'Japanese Restaurant',
 'Kitchen Supply Store',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Lebanese Restaurant',
 'Lounge',
 'Malay Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Modern European R

In [1174]:
airport_restaurants_stripped.head()

,airport_name,Venue Category,rating
0,HartsfieldñJackson Atlanta International Airport,Sushi Restaurant,9.3
1,HartsfieldñJackson Atlanta International Airport,Restaurant,7.5
2,HartsfieldñJackson Atlanta International Airport,Mexican Restaurant,6.9
3,HartsfieldñJackson Atlanta International Airport,Southern / Soul Food Restaurant,7.3
4,Beijing Capital International Airport,Cantonese Restaurant,5.7


In [1199]:
len(airport_restaurants_stripped)

326

In [1175]:
restaurant_df = pd.DataFrame()

In [1176]:
for idx, item in enumerate(airport_restaurants_stripped.iterrows()):
    restaurant_df.loc[idx, item[1][1]] = float(item[1][2])
restaurant_df["airport_name"] = airport_restaurants_stripped["airport_name"]

In [1177]:
restaurant_df.head()

,Sushi Restaurant,Restaurant,Mexican Restaurant,Southern / Soul Food Restaurant,Cantonese Restaurant,Asian Restaurant,Steakhouse,Middle Eastern Restaurant,Lebanese Restaurant,Café,Lounge,Indian Restaurant,Furniture / Home Store,Mediterranean Restaurant,Diner,American Restaurant,Italian Restaurant,Kitchen Supply Store,Japanese Restaurant,French Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Spanish Restaurant,Hawaiian Restaurant,Sports Bar,Greek Restaurant,Brewery,Pub,Breakfast Spot,Hotel Bar,Hong Kong Restaurant,Cha Chaan Teng,Food Court,Vegetarian / Vegan Restaurant,Indonesian Restaurant,Korean Restaurant,Gastropub,Coffee Shop,Dutch Restaurant,Turkish Restaurant,German Restaurant,Malay Restaurant,Salad Place,Dessert Shop,Dim Sum Restaurant,Caribbean Restaurant,Other Nightlife,Scandinavian Restaurant,Halal Restaurant,New American Restaurant,Airport Food Court,Beer Garden,Moroccan Restaurant,Modern European Restaurant,Filipino Restaurant,Pizza Place,Szechuan Restaurant,Food & Drink Shop,Food,Nightclub,Wings Joint,Cuban Restaurant,Latin American Restaurant,South American Restaurant,Fast Food Restaurant,Beer Bar,airport_name
0,9.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HartsfieldñJackson Atlanta International Airport
1,NaN,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HartsfieldñJackson Atlanta International Airport
2,NaN,NaN,6.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HartsfieldñJackson Atlanta International Airport
3,NaN,NaN,NaN,7.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HartsfieldñJackson Atlanta International Airport
4,NaN,NaN,NaN,NaN,5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Beijing Capital International Airport


In [1178]:
restaurant_df_grouped = restaurant_df.groupby("airport_name").mean()

In [1232]:
airport_restaurants_stripped.groupby("airport_name").mean().sort_values(by="rating", ascending=False)

,rating
airport_name,
Dubai International Airport,8.442105
McCarran International Airport,8.378947
Miami International Airport,8.000000
Los Angeles International Airport,7.929412
Istanbul Atat¸rk Airport,7.880000
HartsfieldñJackson Atlanta International Airport,7.750000
Amsterdam Airport Schiphol,7.541176
Tokyo International Airport,7.523529
Charlotte Douglas International Airport,7.476923


In [1179]:
restaurant_df_grouped = restaurant_df_grouped.fillna(0)

In [1227]:
restaurant_df_grouped.head(25)

,airport_name,Sushi Restaurant,Restaurant,Mexican Restaurant,Southern / Soul Food Restaurant,Cantonese Restaurant,Asian Restaurant,Steakhouse,Middle Eastern Restaurant,Lebanese Restaurant,Café,Lounge,Indian Restaurant,Furniture / Home Store,Mediterranean Restaurant,Diner,American Restaurant,Italian Restaurant,Kitchen Supply Store,Japanese Restaurant,French Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Spanish Restaurant,Hawaiian Restaurant,Sports Bar,Greek Restaurant,Brewery,Pub,Breakfast Spot,Hotel Bar,Hong Kong Restaurant,Cha Chaan Teng,Food Court,Vegetarian / Vegan Restaurant,Indonesian Restaurant,Korean Restaurant,Gastropub,Coffee Shop,Dutch Restaurant,Turkish Restaurant,German Restaurant,Malay Restaurant,Salad Place,Dessert Shop,Dim Sum Restaurant,Caribbean Restaurant,Other Nightlife,Scandinavian Restaurant,Halal Restaurant,New American Restaurant,Airport Food Court,Beer Garden,Moroccan Restaurant,Modern European Restaurant,Filipino Restaurant,Pizza Place,Szechuan Restaurant,Food & Drink Shop,Food,Nightclub,Wings Joint,Cuban Restaurant,Latin American Restaurant,South American Restaurant,Fast Food Restaurant,Beer Bar,Cluster Label
0,Amsterdam Airport Schiphol,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,6.733333,0.0,0.0,0.00,6.400,6.20,0.0,6.3,0.0,7.4,7.40,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,5.6,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,6.8,0.0,0.0,6.2,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,6.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,3
1,Beijing Capital International Airport,0.0,0.00,0.00,0.0,5.7,4.9,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.000,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1
2,Charles de Gaulle Airport,0.0,0.00,0.00,0.0,0.0,0.0,5.2,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.000,0.00,0.0,0.0,0.0,0.0,5.80,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1
3,Dallas/Fort Worth International Airport,0.0,0.00,8.40,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,7.60,0.000,5.00,0.0,0.0,0.0,8.0,5.95,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1
4,Denver International Airport,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.000,7.70,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,7.9,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.1,7.65,0.0,0.0,0.0,1
5,Dubai International Airport,0.0,0.00,0.00,0.0,0.0,0.0,0.0,8.22,0.0,0.00,0.0,8.050000,8.1,7.0,7.95,7.700,7.90,6.6,8.7,9.2,7.8,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,3
6,Frankfurt Airport,0.0,8.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,6.900,0.00,0.0,0.0,0.0,6.5,5.80,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.5,0.0,0.0,6.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1
7,Guangzhou Baiyun International Airport,0.0,0.00,0.00,0.0,0.0,0.0,8.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.000,0.00,0.0,0.0,0.0,0.0,6.70,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1
8,HartsfieldñJackson Atlanta International Airport,9.3,7.50,6.90,7.3,0.0,0.0,0.0,0.00,0.0,

In [1181]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(restaurant_df_grouped)

# check cluster labels generated for each row in the dataframe
set(kmeans.labels_)

{0, 1, 2, 3, 4}

In [1182]:
airports_merged = my_data.copy()
len(set(restaurant_df_grouped.index))

20

In [1183]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Best Rated Restaurant Category'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Best Rated Restaurant Category'.format(ind+1))

# create a new dataframe
restaurant_df_sorted = pd.DataFrame(columns=columns)
restaurant_df_sorted['name'] = restaurant_df_grouped.index

for ind in np.arange(restaurant_df_grouped.shape[0]):
    restaurant_df_sorted.iloc[ind, 1:] = return_most_common_venues(restaurant_df_grouped.iloc[ind, :], num_top_venues)

restaurant_df_sorted.head()


,name,1st Best Rated Restaurant Category,2nd Best Rated Restaurant Category,3rd Best Rated Restaurant Category,4th Best Rated Restaurant Category,5th Best Rated Restaurant Category,6th Best Rated Restaurant Category,7th Best Rated Restaurant Category,8th Best Rated Restaurant Category,9th Best Rated Restaurant Category,10th Best Rated Restaurant Category
0,Amsterdam Airport Schiphol,Seafood Restaurant,Chinese Restaurant,Scandinavian Restaurant,Indonesian Restaurant,Indian Restaurant,Halal Restaurant,American Restaurant,Japanese Restaurant,Italian Restaurant,Coffee Shop
1,Beijing Capital International Airport,Cantonese Restaurant,Asian Restaurant,Beer Bar,Spanish Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar
2,Charles de Gaulle Airport,Chinese Restaurant,Steakhouse,Beer Bar,Spanish Restaurant,Seafood Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Hawaiian Restaurant
3,Dallas/Fort Worth International Airport,Mexican Restaurant,Seafood Restaurant,Diner,Chinese Restaurant,Italian Restaurant,Bar,Japanese Curry Restaurant,Thai Restaurant,Buffet,Beer Bar
4,Denver International Airport,Cuban Restaurant,Bar,Italian Restaurant,Latin American Restaurant,Buffet,Japanese Restaurant,French Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant


In [1184]:
restaurant_df_sorted.head(50)

,name,1st Best Rated Restaurant Category,2nd Best Rated Restaurant Category,3rd Best Rated Restaurant Category,4th Best Rated Restaurant Category,5th Best Rated Restaurant Category,6th Best Rated Restaurant Category,7th Best Rated Restaurant Category,8th Best Rated Restaurant Category,9th Best Rated Restaurant Category,10th Best Rated Restaurant Category
0,Amsterdam Airport Schiphol,Seafood Restaurant,Chinese Restaurant,Scandinavian Restaurant,Indonesian Restaurant,Indian Restaurant,Halal Restaurant,American Restaurant,Japanese Restaurant,Italian Restaurant,Coffee Shop
1,Beijing Capital International Airport,Cantonese Restaurant,Asian Restaurant,Beer Bar,Spanish Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar
2,Charles de Gaulle Airport,Chinese Restaurant,Steakhouse,Beer Bar,Spanish Restaurant,Seafood Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Hawaiian Restaurant
3,Dallas/Fort Worth International Airport,Mexican Restaurant,Seafood Restaurant,Diner,Chinese Restaurant,Italian Restaurant,Bar,Japanese Curry Restaurant,Thai Restaurant,Buffet,Beer Bar
4,Denver International Airport,Cuban Restaurant,Bar,Italian Restaurant,Latin American Restaurant,Buffet,Japanese Restaurant,French Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant
5,Dubai International Airport,French Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Furniture / Home Store,Indian Restaurant,Diner,Italian Restaurant,Seafood Restaurant,American Restaurant,Mediterranean Restaurant
6,Frankfurt Airport,Scandinavian Restaurant,Restaurant,American Restaurant,Seafood Restaurant,Airport Food Court,Chinese Restaurant,Pub,Japanese Curry Restaurant,Japanese Restaurant,French Restaurant
7,Guangzhou Baiyun International Airport,Steakhouse,Chinese Restaurant,Beer Bar,Spanish Restaurant,Seafood Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Hawaiian Restaurant
8,HartsfieldñJackson Atlanta International Airport,Restaurant,Southern / Soul Food Restaurant,Mexican Restaurant,Spanish Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar
9,Heathrow Airport,Restaurant,American Restaurant,Asian Restaurant,Hong Kong Restaurant,Pub,Gastropub,Korean Restaurant,Café,Cantonese Restaurant,Chinese Restaurant


In [1185]:
#merge airport data with most best rated venue(restaurant)

airports_merged = airports_merged.join(restaurant_df_sorted.set_index('name'), on='name')

try:
    airports_merged = airports_merged.drop("rating_mean", 1)
except:
    pass

In [1197]:
restaurant_df_grouped["Cluster Label"] = kmeans.labels_

In [1198]:
restaurant_df_grouped.head()

,Sushi Restaurant,Restaurant,Mexican Restaurant,Southern / Soul Food Restaurant,Cantonese Restaurant,Asian Restaurant,Steakhouse,Middle Eastern Restaurant,Lebanese Restaurant,Café,Lounge,Indian Restaurant,Furniture / Home Store,Mediterranean Restaurant,Diner,American Restaurant,Italian Restaurant,Kitchen Supply Store,Japanese Restaurant,French Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Spanish Restaurant,Hawaiian Restaurant,Sports Bar,Greek Restaurant,Brewery,Pub,Breakfast Spot,Hotel Bar,Hong Kong Restaurant,Cha Chaan Teng,Food Court,Vegetarian / Vegan Restaurant,Indonesian Restaurant,Korean Restaurant,Gastropub,Coffee Shop,Dutch Restaurant,Turkish Restaurant,German Restaurant,Malay Restaurant,Salad Place,Dessert Shop,Dim Sum Restaurant,Caribbean Restaurant,Other Nightlife,Scandinavian Restaurant,Halal Restaurant,New American Restaurant,Airport Food Court,Beer Garden,Moroccan Restaurant,Modern European Restaurant,Filipino Restaurant,Pizza Place,Szechuan Restaurant,Food & Drink Shop,Food,Nightclub,Wings Joint,Cuban Restaurant,Latin American Restaurant,South American Restaurant,Fast Food Restaurant,Beer Bar,Cluster Label
airport_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Amsterdam Airport Schiphol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.733333,0.0,0.0,0.0,6.4,6.2,0.0,6.3,0.0,7.4,7.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.8,0.0,0.0,6.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,6.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,3
Beijing Capital International Airport,0.0,0.0,0.0,0.0,5.7,4.9,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1
Charles de Gaulle Airport,0.0,0.0,0.0,0.0,0.0,0.0,5.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1
Dallas/Fort Worth International Airport,0.0,0.0,8.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,7.6,0.0,5.0,0.0,0.0,0.0,8.0,5.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1
Denver International Airport,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,7.7,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,7.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.1,7.65,0.0,0.0,0.0,1


In [1194]:
len(kmeans.labels_)

20

In [1201]:
restaurant_df_grouped.reset_index(level=0, inplace=True)

In [1205]:
airports_merged_wcluster = airports_merged.copy()

In [1215]:
airports_merged_wcluster = airports_merged_wcluster.merge(restaurant_df_grouped, how="right",left_on="name", right_on="airport_name")

In [1187]:
x = np.arange(kclusters)

ys = [i+x+(i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [1221]:
# create map
airports_clusters = folium.Map(location=[51.509865, -0.118092], zoom_start=2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
for lat, lng, label in zip(my_data['latitude'], my_data['longitude'], my_data['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='grey',
        fill=True,
        fill_color='grey',
        fill_opacity=0.7,).add_to(airports_clusters)
    

markers_colors = []
for lat, lon, poi, cluster in zip(airports_merged_wcluster['latitude'], airports_merged_wcluster['longitude'], airports_merged_wcluster['name'], airports_merged_wcluster['Cluster Label']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(airports_clusters)
 
    
    # add some info to the map to get an immediate understanding of the approach
folium.Marker(
    [84, -70],
    icon=DivIcon(
        icon_size=(600,200),
        icon_anchor=(100,0),
        html='<div style="font-size: 14pt; text-align:center">Airports clustered based on their restaurant user ratings to get an overview of performance in this area for airport operators</div>')
    ).add_to(airports_clusters)

airports_clusters

#### The analysis shows there is one large cluster and a few that stand out from the crowd. However, a granular "restaurant analysis" based on the different  restaurant categories is biased because for some airports there are simply not enough or no rated restaurants.

#### Therefore, I will refine the approach to give airport operators or restaurant owners an unbiased impression how they rank among the top 50 airports and identify the "restaurant profile" for each airport and use it to cluster the airports.

### 3.2. Restaurant Types

In [982]:
def getNearbyVenues_explore(names, latitudes, longitudes, radius=8000, LIMIT_to=50, query_term="restaurant"):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        try:    
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
      
        except:
            venues_list.append([(
            name, 
            lat, 
            lng, 
            0, 
            0, 
            0,  
            0) for v in results])
        

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['name', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [983]:
airport_restaurants_full = getNearbyVenues_explore(names=my_data['name'],
                                   latitudes=my_data['latitude'],
                                   longitudes=my_data['longitude'],
                                   radius= 10000,
                                   LIMIT_to=50,
                                   query_term="restaurant"
                                  )

HartsfieldñJackson Atlanta International Airport
Beijing Capital International Airport
Dubai International Airport
Los Angeles International Airport
Tokyo International Airport
O'Hare International Airport
Heathrow Airport
Hong Kong International Airport
Shanghai Pudong International Airport
Charles de Gaulle Airport
Dallas/Fort Worth International Airport
Amsterdam Airport Schiphol
Frankfurt Airport
Istanbul Atat¸rk Airport
Guangzhou Baiyun International Airport
John F. Kennedy International Airport
Singapore Changi Airport
Denver International Airport
Seoul Incheon International Airport
Suvarnabhumi Airport
Indira Gandhi International Airport
SoekarnoñHatta International Airport
San Francisco International Airport
Kuala Lumpur International Airport
Madrid Barajas Airport
McCarran International Airport
Chengdu Shuangliu International Airport
Seattle-Tacoma International Airport
Chhatrapati Shivaji International Airport
Miami International Airport
Charlotte Douglas International Airpor

In [984]:
airport_restaurants_full.head()

,name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,HartsfieldñJackson Atlanta International Airport,33.636719,-84.428067,One Flew South Restaurant & Sushi Bar,33.640935,-84.425910,Sushi Restaurant
1,HartsfieldñJackson Atlanta International Airport,33.636719,-84.428067,Chicken & Beer,33.638494,-84.429318,Fried Chicken Joint
2,HartsfieldñJackson Atlanta International Airport,33.636719,-84.428067,Delta Sky Club,33.641033,-84.435977,Airport Lounge
3,HartsfieldñJackson Atlanta International Airport,33.636719,-84.428067,Bobby's Burger Palace,33.642315,-84.435974,American Restaurant
4,HartsfieldñJackson Atlanta International Airport,33.636719,-84.428067,Fresh Healthy Cafe,33.642733,-84.432595,Snack Place


In [985]:
airport_restaurants_full.shape

(4645, 7)

In [986]:
len(set(airport_restaurants_full["Venue Category"]))

379

In [987]:
# one hot encoding
airport_onehot = pd.get_dummies(airport_restaurants_full[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
airport_onehot['name'] = airport_restaurants_full["name"] 

# move neighborhood column to the first column
fixed_columns = [airport_onehot.columns[-1]] + list(airport_onehot.columns[:-1])
airport_onehot = airport_onehot[fixed_columns]

airport_onehot.head()

,name,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bike Trail,Bistro,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Champagne Bar,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Restaurant,Chocolate Shop,Church,City,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Goan Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Meatball Place,Indonesian Restaurant,Irani Cafe,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karnataka Restaurant,Kids Store,Kofte Place,Korean Restaurant,Kuruyemişçi,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outlet Mall,Outlet Store,Paella Restaurant,Pakistani Restaurant,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Private School,Pub,Public Art,Public Bathroom,Racetrack,Rafting,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Dec

In [988]:
airport_restaurants_grouped = airport_onehot.groupby('name').mean().reset_index()
airport_restaurants_grouped.head()

,name,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bike Trail,Bistro,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Champagne Bar,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Restaurant,Chocolate Shop,Church,City,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Goan Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Meatball Place,Indonesian Restaurant,Irani Cafe,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karnataka Restaurant,Kids Store,Kofte Place,Korean Restaurant,Kuruyemişçi,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outlet Mall,Outlet Store,Paella Restaurant,Pakistani Restaurant,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Private School,Pub,Public Art,Public Bathroom,Racetrack,Rafting,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Dec

In [989]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [990]:
num_top_venues = 50

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
airport_restaurants_sorted = pd.DataFrame(columns=columns)
airport_restaurants_sorted['name'] = airport_restaurants_grouped['name']

for ind in np.arange(airport_restaurants_grouped.shape[0]):
    airport_restaurants_sorted.iloc[ind, 1:] = return_most_common_venues(airport_restaurants_grouped.iloc[ind, :], num_top_venues)

airport_restaurants_sorted.head()

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
0,Amsterdam Airport Schiphol,Hotel,Restaurant,Park,Indonesian Restaurant,Music Venue,Gym,Athletics & Sports,Coffee Shop,Nightclub,Shopping Mall,Café,Burger Joint,Supermarket,Golf Course,French Restaurant,Ice Cream Shop,Fish Market,Market,Comedy Club,Tech Startup,Ethiopian Restaurant,Yoga Studio,Beer Bar,Hotel Bar,Cheese Shop,Airport Lounge,Marijuana Dispensary,Movie Theater,Garden,Lake,Food Service,Furniture / Home Store,Organic Grocery,Farm,Pizza Place,Chocolate Shop,Forest,Office,Arcade,Spa,Comfort Food Restaurant,Concert Hall,Brewery,Pub,Diner,Campground,Department Store,Hobby Shop,Italian Restaurant,Turkish Restaurant
1,BarcelonañEl Prat Airport,Tapas Restaurant,Pizza Place,Park,Spanish Restaurant,Restaurant,Gym / Fitness Center,Mediterranean Restaurant,Burger Joint,Hotel,Beach,Italian Restaurant,Sporting Goods Shop,Deli / Bodega,Seafood Restaurant,Athletics & Sports,Basketball Stadium,Cocktail Bar,Tennis Court,Boarding House,Soccer Stadium,Stadium,Sports Club,Bistro,Football Stadium,Spa,Liquor Store,South American Restaurant,Playground,Mountain,Scenic Lookout,Pharmacy,Dessert Shop,Coffee Shop,Ramen Restaurant,Castle,Candy Store,Nature Preserve,Snack Place,Café,Farmers Market,Shopping Mall,Museum,Tea Room,Plaza,Multiplex,Mexican Restaurant,Gym,Bakery,Japanese Restaurant,Hostel
2,Beijing Capital International Airport,Coffee Shop,Hotel,Café,Fast Food Restaurant,Chinese Restaurant,Italian Restaurant,Airport Lounge,Thai Restaurant,Noodle House,Hotel Bar,Supermarket,American Restaurant,Burger Joint,Cantonese Restaurant,Shopping Mall,Bar,Pizza Place,Airport Service,Hot Spring,Ice Cream Shop,Hong Kong Restaurant,Convenience Store,Korean Restaurant,Resort,Ramen Restaurant,Farm,Japanese Restaurant,New American Restaurant,French Restaurant,Metro Station,Malay Restaurant,Lake,Plaza,Restaurant,Asian Restaurant,Bagel Shop,Art Gallery,Bakery,Airport,Argentinian Restaurant,Toll Booth,Fried Chicken Joint,Food Court,Flower Shop,Food,Arepa Restaurant,Gaming Cafe,Furniture / Home Store,Food & Drink Shop,Food Service
3,Benito Ju·rez International Airport,Mexican Restaurant,Art Museum,Taco Place,Hotel,History Museum,Ice Cream Shop,Bakery,Plaza,Museum,Stadium,Spanish Restaurant,Coffee Shop,Concert Hall,Juice Bar,Movie Theater,Farmers Market,Historic Site,Breakfast Spot,Jazz Club,Building,Burger Joint,Soccer Field,Candy Store,Scenic Lookout,Chocolate Shop,Sandwich Place,Church,Clothing Store,Middle Eastern Restaurant,Restaurant,Racetrack,Deli / Bodega,Department Store,Dessert Shop,Post Office,Motel,Big Box Store,Pet Store,Park,Other Great Outdoors,Mountain,Opera House,Falafel Restaurant,Non-Profit,Pizza Place,Hostel,Baseball Stadium,Airport Lounge,Theater,Bed & Breakfast
4,Charles de Gaulle Airport,Hotel,Airport Lounge,Dessert Shop,Coffee Shop,French Restaurant,Bakery,Sandwich Place,Chocolate Shop,Multiplex,Cosmetics Sho

In [991]:
# set number of clusters
kclusters = 5

airport_restaurants_clustering = airport_restaurants_grouped.drop('name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(airport_restaurants_clustering)

# check cluster labels generated for each row in the dataframe
set(kmeans.labels_)

{0, 1, 2, 3, 4}

In [992]:
# add clustering labels
my_data_restaurants = my_data.copy()
my_data_restaurants['Cluster Labels'] = kmeans.labels_

#merge airport data with most frequent venue(restaurant) to add latitude/longitude for each airport

my_data_restaurants = my_data_restaurants.join(airport_restaurants_sorted.set_index('name'), on='name')

try:
    my_data_restaurants = my_data_restaurants.drop("rating_mean", 1)
except:
    pass
my_data_restaurants.head()

,name,location,country,city,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
0,HartsfieldñJackson Atlanta International Airport,"Atlanta, Georgia",United States,Atlanta,33.636719,-84.428067,1,Fast Food Restaurant,Airport Lounge,Hotel,Caribbean Restaurant,Sandwich Place,Breakfast Spot,Restaurant,Pizza Place,Burger Joint,Art Gallery,Chinese Restaurant,Burrito Place,Museum,Coffee Shop,Greek Restaurant,BBQ Joint,Grocery Store,Italian Restaurant,Gym / Fitness Center,Café,Cosmetics Shop,Wings Joint,Park,Pet Store,Beer Bar,Ethiopian Restaurant,Sushi Restaurant,Lounge,Frozen Yogurt Shop,Steakhouse,Fried Chicken Joint,Seafood Restaurant,Airport,Roof Deck,Southern / Soul Food Restaurant,Brewery,Motel,African Restaurant,Pharmacy,Snack Place,Smoothie Shop,Cajun / Creole Restaurant,Gas Station,Taco Place,Thai Restaurant,Travel Lounge,Parking,Pub,Vietnamese Restaurant,Bar
1,Beijing Capital International Airport,"Chaoyang-Shunyi, Beijing",China,Beijing,40.080111,116.584556,1,Coffee Shop,Hotel,Café,Fast Food Restaurant,Chinese Restaurant,Italian Restaurant,Airport Lounge,Thai Restaurant,Noodle House,Hotel Bar,Supermarket,American Restaurant,Burger Joint,Cantonese Restaurant,Shopping Mall,Bar,Pizza Place,Airport Service,Hot Spring,Ice Cream Shop,Hong Kong Restaurant,Convenience Store,Korean Restaurant,Resort,Ramen Restaurant,Farm,Japanese Restaurant,New American Restaurant,French Restaurant,Metro Station,Malay Restaurant,Lake,Plaza,Restaurant,Asian Restaurant,Bagel Shop,Art Gallery,Bakery,Airport,Argentinian Restaurant,Toll Booth,Fried Chicken Joint,Food Court,Flower Shop,Food,Arepa Restaurant,Gaming Cafe,Furniture / Home Store,Food & Drink Shop,Food Service
2,Dubai International Airport,"Garhoud, Dubai",United Arab Emirates,Dubai,25.252778,55.364444,2,Coffee Shop,Hotel,Middle Eastern Restaurant,Airport Lounge,Lounge,Seafood Restaurant,Café,Shopping Mall,Italian Restaurant,Indian Restaurant,American Restaurant,Golf Course,Clothing Store,Mediterranean Restaurant,Restaurant,Residential Building (Apartment / Condo),Comfort Food Restaurant,Record Shop,Zoo,Sandwich Place,Rock Club,Department Store,Chinese Restaurant,Spa,Pub,French Restaurant,Duty-free Shop,Park,Nightclub,Multiplex,Tennis Stadium,Frozen Yogurt Shop,Furniture / Home Store,Lebanese Restaurant,Korean Restaurant,Juice Bar,Gym,Irish Pub,Ice Cream Shop,Hotel Pool,Buffet,Resort,Thai Restaurant,Airport,Asian Restaurant,Bar,Food,Food & Drink Shop,Food Court,Food Service
3,Los Angeles International Airport,"Los Angeles, California",United States,Los Angeles,33.942536,-118.408075,1,Grocery Store,American Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,Beach,Bakery,Italian Restaurant,Mexican Restaurant,Restaurant,Breakfast Spot,Gym,Dog Run,Wine Bar,Japanese Restaurant,New American Restaurant,Burger Joint,Yoga Studio,Movie Theater,Brewery,Park,Diner,Furniture / Home Store,Seafood R

In [993]:
x = np.arange(kclusters)

ys = [i+x+(i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [1235]:
# create map
airport_clusters = folium.Map(location=[51.509865, -0.118092], zoom_start=2)

from folium.features import DivIcon

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(my_data_restaurants['latitude'], my_data_restaurants['longitude'], my_data_restaurants['name'], my_data_restaurants['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(airport_clusters)
    
# add some info to the map to get an immediate understanding of the approach
folium.Marker(
    [84, -70],
    icon=DivIcon(
        icon_size=(600,200),
        icon_anchor=(100,0),
        html='<div style="font-size: 14pt; text-align:center">Airports clustered based on their restaurant profile to identify gaps for airport operators or restaurants chains / owners</div>')
    ).add_to(airport_clusters)

airport_clusters

### 4. Examine Clusters

#### 4.1. Restaurant Quality

In [1222]:
airports_merged_wcluster.loc[airports_merged_wcluster['Cluster Label'] == 0, airports_merged_wcluster.columns[[0] + [2] + list(range(6, airports_merged_wcluster.shape[1]))]].head(10)

,name,country,1st Best Rated Restaurant Category,2nd Best Rated Restaurant Category,3rd Best Rated Restaurant Category,4th Best Rated Restaurant Category,5th Best Rated Restaurant Category,6th Best Rated Restaurant Category,7th Best Rated Restaurant Category,8th Best Rated Restaurant Category,9th Best Rated Restaurant Category,10th Best Rated Restaurant Category,airport_name,Sushi Restaurant,Restaurant,Mexican Restaurant,Southern / Soul Food Restaurant,Cantonese Restaurant,Asian Restaurant,Steakhouse,Middle Eastern Restaurant,Lebanese Restaurant,Café,Lounge,Indian Restaurant,Furniture / Home Store,Mediterranean Restaurant,Diner,American Restaurant,Italian Restaurant,Kitchen Supply Store,Japanese Restaurant,French Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Spanish Restaurant,Hawaiian Restaurant,Sports Bar,Greek Restaurant,Brewery,Pub,Breakfast Spot,Hotel Bar,Hong Kong Restaurant,Cha Chaan Teng,Food Court,Vegetarian / Vegan Restaurant,Indonesian Restaurant,Korean Restaurant,Gastropub,Coffee Shop,Dutch Restaurant,Turkish Restaurant,German Restaurant,Malay Restaurant,Salad Place,Dessert Shop,Dim Sum Restaurant,Caribbean Restaurant,Other Nightlife,Scandinavian Restaurant,Halal Restaurant,New American Restaurant,Airport Food Court,Beer Garden,Moroccan Restaurant,Modern European Restaurant,Filipino Restaurant,Pizza Place,Szechuan Restaurant,Food & Drink Shop,Food,Nightclub,Wings Joint,Cuban Restaurant,Latin American Restaurant,South American Restaurant,Fast Food Restaurant,Beer Bar,Cluster Label
15,John F. Kennedy International Airport,United States,New American Restaurant,Mexican Restaurant,Diner,French Restaurant,Japanese Restaurant,Italian Restaurant,Caribbean Restaurant,Breakfast Spot,Kitchen Supply Store,American Restaurant,John F. Kennedy International Airport,8.1,0.0,8.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.9,7.65,8.65,8.2,8.7,8.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.6,0.0,6.8,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [1223]:
airports_merged_wcluster.loc[airports_merged_wcluster['Cluster Label'] == 1, airports_merged_wcluster.columns[[0] + [2] + list(range(6, airports_merged_wcluster.shape[1]))]].head(10)

,name,country,1st Best Rated Restaurant Category,2nd Best Rated Restaurant Category,3rd Best Rated Restaurant Category,4th Best Rated Restaurant Category,5th Best Rated Restaurant Category,6th Best Rated Restaurant Category,7th Best Rated Restaurant Category,8th Best Rated Restaurant Category,9th Best Rated Restaurant Category,10th Best Rated Restaurant Category,airport_name,Sushi Restaurant,Restaurant,Mexican Restaurant,Southern / Soul Food Restaurant,Cantonese Restaurant,Asian Restaurant,Steakhouse,Middle Eastern Restaurant,Lebanese Restaurant,Café,Lounge,Indian Restaurant,Furniture / Home Store,Mediterranean Restaurant,Diner,American Restaurant,Italian Restaurant,Kitchen Supply Store,Japanese Restaurant,French Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Spanish Restaurant,Hawaiian Restaurant,Sports Bar,Greek Restaurant,Brewery,Pub,Breakfast Spot,Hotel Bar,Hong Kong Restaurant,Cha Chaan Teng,Food Court,Vegetarian / Vegan Restaurant,Indonesian Restaurant,Korean Restaurant,Gastropub,Coffee Shop,Dutch Restaurant,Turkish Restaurant,German Restaurant,Malay Restaurant,Salad Place,Dessert Shop,Dim Sum Restaurant,Caribbean Restaurant,Other Nightlife,Scandinavian Restaurant,Halal Restaurant,New American Restaurant,Airport Food Court,Beer Garden,Moroccan Restaurant,Modern European Restaurant,Filipino Restaurant,Pizza Place,Szechuan Restaurant,Food & Drink Shop,Food,Nightclub,Wings Joint,Cuban Restaurant,Latin American Restaurant,South American Restaurant,Fast Food Restaurant,Beer Bar,Cluster Label
0,HartsfieldñJackson Atlanta International Airport,United States,Restaurant,Southern / Soul Food Restaurant,Mexican Restaurant,Spanish Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,HartsfieldñJackson Atlanta International Airport,9.3,7.50,6.90,7.3,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,Beijing Capital International Airport,China,Cantonese Restaurant,Asian Restaurant,Beer Bar,Spanish Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Beijing Capital International Airport,0.0,0.00,0.00,0.0,5.7,4.9,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6,Heathrow Airport,United Kingdom,Restaurant,American Restaurant,Asian Restaurant,Hong Kong Restaurant,Pub,Gastropub,Korean Restaurant,Café,Cantonese Restaurant,Chinese Restaurant,Heathrow Airport,0.0,7.80,0.00,0.0,0.0,7.1,0.0,0.0,0.0,4.50,0.0,0.00,0.0,0.0,0.0,7.70,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,0.0,0.0,6.9,0.0,0.0,0.0,0.0,5.3,5.6,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,Hong Kong International Airport,China,French Restaurant,Dutch Restaurant,Restaurant,Café,Turkish Restaurant,German Restaurant,Greek Restaurant,Sports Bar,Hawaiian Restaurant,Spanish Restaurant,Hong Kong International Airport,0.0,7.95,0.00,0.0,0.0,0.0,0.0,0.0,0.0,7.90,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,8.3,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.1,7.45,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,Shanghai Pudong International Airport,China,Café,Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Spanish Restaurant,Japanese Restaurant,Shanghai Pudong International Airport,0.0,8.20,0.00,0.0,0.0,0.0,0.0,0.0,0.0,8.26,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0

In [1224]:
airports_merged_wcluster.loc[airports_merged_wcluster['Cluster Label'] == 2, airports_merged_wcluster.columns[[0] + [2] + list(range(6, airports_merged_wcluster.shape[1]))]].head(10)

,name,country,1st Best Rated Restaurant Category,2nd Best Rated Restaurant Category,3rd Best Rated Restaurant Category,4th Best Rated Restaurant Category,5th Best Rated Restaurant Category,6th Best Rated Restaurant Category,7th Best Rated Restaurant Category,8th Best Rated Restaurant Category,9th Best Rated Restaurant Category,10th Best Rated Restaurant Category,airport_name,Sushi Restaurant,Restaurant,Mexican Restaurant,Southern / Soul Food Restaurant,Cantonese Restaurant,Asian Restaurant,Steakhouse,Middle Eastern Restaurant,Lebanese Restaurant,Café,Lounge,Indian Restaurant,Furniture / Home Store,Mediterranean Restaurant,Diner,American Restaurant,Italian Restaurant,Kitchen Supply Store,Japanese Restaurant,French Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Spanish Restaurant,Hawaiian Restaurant,Sports Bar,Greek Restaurant,Brewery,Pub,Breakfast Spot,Hotel Bar,Hong Kong Restaurant,Cha Chaan Teng,Food Court,Vegetarian / Vegan Restaurant,Indonesian Restaurant,Korean Restaurant,Gastropub,Coffee Shop,Dutch Restaurant,Turkish Restaurant,German Restaurant,Malay Restaurant,Salad Place,Dessert Shop,Dim Sum Restaurant,Caribbean Restaurant,Other Nightlife,Scandinavian Restaurant,Halal Restaurant,New American Restaurant,Airport Food Court,Beer Garden,Moroccan Restaurant,Modern European Restaurant,Filipino Restaurant,Pizza Place,Szechuan Restaurant,Food & Drink Shop,Food,Nightclub,Wings Joint,Cuban Restaurant,Latin American Restaurant,South American Restaurant,Fast Food Restaurant,Beer Bar,Cluster Label
3,Los Angeles International Airport,United States,Indian Restaurant,Spanish Restaurant,Hawaiian Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Restaurant,Thai Restaurant,Japanese Curry Restaurant,Buffet,Los Angeles International Airport,0.0,7.85,8.1,0.0,0.0,0.0,0.0,6.7,0.0,0.0,0.0,8.55,0.0,0.0,7.6,0.0,8.5,0.0,8.4,0.0,7.7,5.2,7.8,7.8,7.75,0.0,8.5,8.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [1225]:
airports_merged_wcluster.loc[airports_merged_wcluster['Cluster Label'] == 3, airports_merged_wcluster.columns[[0] + [2] + list(range(6, airports_merged_wcluster.shape[1]))]].head(10)

,name,country,1st Best Rated Restaurant Category,2nd Best Rated Restaurant Category,3rd Best Rated Restaurant Category,4th Best Rated Restaurant Category,5th Best Rated Restaurant Category,6th Best Rated Restaurant Category,7th Best Rated Restaurant Category,8th Best Rated Restaurant Category,9th Best Rated Restaurant Category,10th Best Rated Restaurant Category,airport_name,Sushi Restaurant,Restaurant,Mexican Restaurant,Southern / Soul Food Restaurant,Cantonese Restaurant,Asian Restaurant,Steakhouse,Middle Eastern Restaurant,Lebanese Restaurant,Café,Lounge,Indian Restaurant,Furniture / Home Store,Mediterranean Restaurant,Diner,American Restaurant,Italian Restaurant,Kitchen Supply Store,Japanese Restaurant,French Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Spanish Restaurant,Hawaiian Restaurant,Sports Bar,Greek Restaurant,Brewery,Pub,Breakfast Spot,Hotel Bar,Hong Kong Restaurant,Cha Chaan Teng,Food Court,Vegetarian / Vegan Restaurant,Indonesian Restaurant,Korean Restaurant,Gastropub,Coffee Shop,Dutch Restaurant,Turkish Restaurant,German Restaurant,Malay Restaurant,Salad Place,Dessert Shop,Dim Sum Restaurant,Caribbean Restaurant,Other Nightlife,Scandinavian Restaurant,Halal Restaurant,New American Restaurant,Airport Food Court,Beer Garden,Moroccan Restaurant,Modern European Restaurant,Filipino Restaurant,Pizza Place,Szechuan Restaurant,Food & Drink Shop,Food,Nightclub,Wings Joint,Cuban Restaurant,Latin American Restaurant,South American Restaurant,Fast Food Restaurant,Beer Bar,Cluster Label
2,Dubai International Airport,United Arab Emirates,French Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Furniture / Home Store,Indian Restaurant,Diner,Italian Restaurant,Seafood Restaurant,American Restaurant,Mediterranean Restaurant,Dubai International Airport,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.22,0.0,0.0,0.0,8.050000,8.1,7.0,7.95,7.700,7.9,6.6,8.7,9.2,7.8,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
5,O'Hare International Airport,United States,Pub,Indian Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Hong Kong Restaurant,Indonesian Restaurant,Cantonese Restaurant,Restaurant,Chinese Restaurant,Italian Restaurant,O'Hare International Airport,0.0,6.3,0.0,0.0,6.5,0.0,0.0,0.00,0.0,0.0,0.0,7.900000,0.0,0.0,0.00,0.000,6.1,0.0,0.0,0.0,7.3,6.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.3,0.0,0.0,7.033333,0.0,0.0,7.9,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
11,Amsterdam Airport Schiphol,The Netherlands,Seafood Restaurant,Chinese Restaurant,Scandinavian Restaurant,Indonesian Restaurant,Indian Restaurant,Halal Restaurant,American Restaurant,Japanese Restaurant,Italian Restaurant,Coffee Shop,Amsterdam Airport Schiphol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,6.733333,0.0,0.0,0.00,6.400,6.2,0.0,6.3,0.0,7.4,7.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.6,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,6.8,0.0,0.0,6.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,6.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
13,Istanbul Atat¸rk Airport,Turkey,Scandinavian Restaurant,Pizza Place,Indian Restaurant,Modern European Restaurant,Filipino Restaurant,Japanese Restaurant,American Restaurant,Chinese Restaurant,Diner,Mexican Restaurant,Istanbul Atat¸rk Airport,0.0,0.0,6.8,0.0,6.2,0.0,0.0,0.00,0.0,0.0,0.0,7.300000,0.0,0.0,6.90,7.075,0.0,0.0,7.1,0.0,5.8,6.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.6,0.0,0.0,0.0,0.0,0.0,7.3,7.3,8.4,6.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [1233]:
airports_merged_wcluster.loc[airports_merged_wcluster['Cluster Label'] == 4, airports_merged_wcluster.columns[[0] + [2] + list(range(6, airports_merged_wcluster.shape[1]))]].head(10)

,name,country,1st Best Rated Restaurant Category,2nd Best Rated Restaurant Category,3rd Best Rated Restaurant Category,4th Best Rated Restaurant Category,5th Best Rated Restaurant Category,6th Best Rated Restaurant Category,7th Best Rated Restaurant Category,8th Best Rated Restaurant Category,9th Best Rated Restaurant Category,10th Best Rated Restaurant Category,airport_name,Sushi Restaurant,Restaurant,Mexican Restaurant,Southern / Soul Food Restaurant,Cantonese Restaurant,Asian Restaurant,Steakhouse,Middle Eastern Restaurant,Lebanese Restaurant,Café,Lounge,Indian Restaurant,Furniture / Home Store,Mediterranean Restaurant,Diner,American Restaurant,Italian Restaurant,Kitchen Supply Store,Japanese Restaurant,French Restaurant,Seafood Restaurant,Chinese Restaurant,Japanese Curry Restaurant,Thai Restaurant,Buffet,Bar,Spanish Restaurant,Hawaiian Restaurant,Sports Bar,Greek Restaurant,Brewery,Pub,Breakfast Spot,Hotel Bar,Hong Kong Restaurant,Cha Chaan Teng,Food Court,Vegetarian / Vegan Restaurant,Indonesian Restaurant,Korean Restaurant,Gastropub,Coffee Shop,Dutch Restaurant,Turkish Restaurant,German Restaurant,Malay Restaurant,Salad Place,Dessert Shop,Dim Sum Restaurant,Caribbean Restaurant,Other Nightlife,Scandinavian Restaurant,Halal Restaurant,New American Restaurant,Airport Food Court,Beer Garden,Moroccan Restaurant,Modern European Restaurant,Filipino Restaurant,Pizza Place,Szechuan Restaurant,Food & Drink Shop,Food,Nightclub,Wings Joint,Cuban Restaurant,Latin American Restaurant,South American Restaurant,Fast Food Restaurant,Beer Bar,Cluster Label
4,Tokyo International Airport,Japan,Breakfast Spot,Mexican Restaurant,American Restaurant,Mediterranean Restaurant,Brewery,Diner,Hotel Bar,Italian Restaurant,Greek Restaurant,Pub,Tokyo International Airport,0.0,0.0,8.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.4,6.8,7.450,6.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.6,0.0,0.0,5.1,5.85,7.2,5.7,8.6,6.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
19,Suvarnabhumi Airport,Thailand,New American Restaurant,Mediterranean Restaurant,Japanese Restaurant,Brewery,Diner,American Restaurant,Pub,Dim Sum Restaurant,Spanish Restaurant,Beer Bar,Suvarnabhumi Airport,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.4,8.2,6.925,0.0,0.0,8.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.00,8.2,6.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.1,0.0,0.0,0.0,0.0,8.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.8,4


In [1234]:
airport_restaurants_stripped.groupby("airport_name").mean().sort_values(by="rating", ascending=False)

,rating
airport_name,
Dubai International Airport,8.442105
McCarran International Airport,8.378947
Miami International Airport,8.000000
Los Angeles International Airport,7.929412
Istanbul Atat¸rk Airport,7.880000
HartsfieldñJackson Atlanta International Airport,7.750000
Amsterdam Airport Schiphol,7.541176
Tokyo International Airport,7.523529
Charlotte Douglas International Airport,7.476923


In [1238]:
airport_restaurants_stripped.sort_values(by="rating").head()

,airport_name,Venue Category,rating
152,Shanghai Pudong International Airport,Café,4.5
146,Shanghai Pudong International Airport,American Restaurant,4.7
306,Suvarnabhumi Airport,Thai Restaurant,4.9
5,Beijing Capital International Airport,Asian Restaurant,4.9
438,Seattle-Tacoma International Airport,Sports Bar,5.0


#### 4.2. Restaurant Profile

In [1239]:
def most_often_in_t10(df):
    counter_dict2 = {}
    for idx, item in enumerate(df.iterrows()):
        weight = 10 ## to account for 1st, 2nd, 3rd etc.
        for i in range(3,13):
            if item[1][i] in counter_dict2:
                counter_dict2[item[1][i]] += (1*weight)
            else:
                counter_dict2[item[1][i]] = (1*weight)
            weight -= 1
    return round(pd.Series(counter_dict2).sort_values(ascending=False)/len(df), 1).head(15)

In [1240]:
my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 0, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]].head(3)

,name,country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Guangzhou Baiyun International Airport,China,0,Hotel,Fast Food Restaurant,Coffee Shop,Airport Lounge,Restaurant,Metro Station,Toll Booth,Cantonese Restaurant,Chinese Restaurant,Asian Restaurant
23,Kuala Lumpur International Airport,Malaysia,0,Coffee Shop,Racetrack,Malay Restaurant,Café,Airport Lounge,Ice Cream Shop,Clothing Store,Asian Restaurant,Boutique,Hotel
42,George Bush Intercontinental Airport,United States,0,Coffee Shop,Mexican Restaurant,Airport Lounge,Hotel,Rental Car Location,Fast Food Restaurant,Pizza Place,Sandwich Place,Furniture / Home Store,Department Store


In [1241]:
len(my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 0, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]])

3

In [1242]:
most_often_in_t10(my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 0, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]])

Coffee Shop             9.3
Airport Lounge          7.0
Hotel                   6.0
Fast Food Restaurant    4.7
Mexican Restaurant      3.0
Racetrack               3.0
Malay Restaurant        2.7
Café                    2.3
Restaurant              2.0
Rental Car Location     2.0
Ice Cream Shop          1.7
Metro Station           1.7
Clothing Store          1.3
Asian Restaurant        1.3
Toll Booth              1.3
dtype: float64

In [1243]:
my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 1, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]].head(3)

,name,country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,HartsfieldñJackson Atlanta International Airport,United States,1,Fast Food Restaurant,Airport Lounge,Hotel,Caribbean Restaurant,Sandwich Place,Breakfast Spot,Restaurant,Pizza Place,Burger Joint,Art Gallery
1,Beijing Capital International Airport,China,1,Coffee Shop,Hotel,Café,Fast Food Restaurant,Chinese Restaurant,Italian Restaurant,Airport Lounge,Thai Restaurant,Noodle House,Hotel Bar
3,Los Angeles International Airport,United States,1,Grocery Store,American Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,Beach,Bakery,Italian Restaurant,Mexican Restaurant,Restaurant,Breakfast Spot


In [1244]:
len(my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 1, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]])

29

In [1245]:
most_often_in_t10(my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 1, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]])

Hotel                   5.9
Coffee Shop             5.2
Airport Lounge          3.8
Café                    3.0
Italian Restaurant      1.9
Pizza Place             1.7
Mexican Restaurant      1.6
Bakery                  1.4
Sandwich Place          1.3
Fast Food Restaurant    1.2
Shopping Mall           1.1
Beach                   1.0
Gym / Fitness Center    1.0
Dessert Shop            1.0
American Restaurant     0.9
dtype: float64

In [1246]:
my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 2, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]].head(3)

,name,country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Dubai International Airport,United Arab Emirates,2,Coffee Shop,Hotel,Middle Eastern Restaurant,Airport Lounge,Lounge,Seafood Restaurant,Café,Shopping Mall,Italian Restaurant,Indian Restaurant
4,Tokyo International Airport,Japan,2,Airport Lounge,Ramen Restaurant,Park,Tonkatsu Restaurant,Hot Spring,Sake Bar,Coffee Shop,Chinese Restaurant,Scenic Lookout,Café
10,Dallas/Fort Worth International Airport,United States,2,Mexican Restaurant,Burger Joint,Coffee Shop,Pizza Place,BBQ Joint,Sandwich Place,Seafood Restaurant,Fast Food Restaurant,Bakery,American Restaurant


In [1247]:
len(my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 2, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]])

14

In [1248]:
most_often_in_t10(my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 2, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]])

Coffee Shop             5.5
Hotel                   5.1
Airport Lounge          4.1
Park                    3.4
Pizza Place             2.7
Mexican Restaurant      1.6
Burger Joint            1.4
BBQ Joint               1.4
Fast Food Restaurant    1.4
Brewery                 1.4
Bakery                  1.4
Ramen Restaurant        1.1
Grocery Store           1.1
Japanese Restaurant     1.1
Bar                     0.9
dtype: float64

In [1249]:
my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 3, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]].head(10)

,name,country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,SoekarnoñHatta International Airport,Indonesia,3,Coffee Shop,Noodle House,Fast Food Restaurant,Asian Restaurant,Indonesian Restaurant,Golf Course,Airport Lounge,Café,Train Station,Pizza Place
24,Madrid Barajas Airport,Spain,3,Spanish Restaurant,Pizza Place,Park,Tapas Restaurant,Restaurant,Airport Lounge,Gym,Italian Restaurant,Athletics & Sports,Grocery Store


In [1250]:
len(my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 3, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]])

2

In [1251]:
most_often_in_t10(my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 3, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]])

Coffee Shop              5.0
Spanish Restaurant       5.0
Pizza Place              5.0
Noodle House             4.5
Airport Lounge           4.5
Fast Food Restaurant     4.0
Park                     4.0
Tapas Restaurant         3.5
Asian Restaurant         3.5
Restaurant               3.0
Indonesian Restaurant    3.0
Golf Course              2.5
Gym                      2.0
Italian Restaurant       1.5
Café                     1.5
dtype: float64

In [1252]:
my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 4, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]].head(10)

,name,country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Heathrow Airport,United Kingdom,4,Airport Lounge,Indian Restaurant,Pub,Coffee Shop,Hotel,Restaurant,Park,Seafood Restaurant,Clothing Store,Airport Service
41,Leonardo da VinciñFiumicino Airport,Italy,4,Beach,Seafood Restaurant,Pizza Place,Ice Cream Shop,Italian Restaurant,Café,Airport Lounge,Restaurant,Historic Site,Food


In [1253]:
len(my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 4, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]])

2

In [1254]:
most_often_in_t10(my_data_restaurants.loc[my_data_restaurants['Cluster Labels'] == 4, my_data_restaurants.columns[[0] + [2] + list(range(6, 17))]])

Airport Lounge        7.0
Seafood Restaurant    6.0
Beach                 5.0
Indian Restaurant     4.5
Pizza Place           4.0
Pub                   4.0
Restaurant            4.0
Coffee Shop           3.5
Ice Cream Shop        3.5
Italian Restaurant    3.0
Hotel                 3.0
Café                  2.5
Park                  2.0
Clothing Store        1.0
Historic Site         1.0
dtype: float64

## 5. Report

### 5.1 Introduction

While flying becomes a commodity, passenger traffic rises and ever more people spent time at and around big airports the business opportunities at airports are still large. Apart from internal transport infrastruce, connecting public transport, new amenities such as sleep pods, sports etc also restaurants are stil a great opportunity to differentiate. 

The data analysis undertaken here is important for airport operators as a means to compare themselves with other top airports across the world but also for restaurant chains and owners to identify coverage gaps or shortfalls in high quality restaurants at certain airports. This analysis can support the optimization of an airports restaurant portfolio or help deciding on where to open a new restaurant and, importantly, in which category (e.g. shushi, fastfood, german).

The following analysis will provide a starting point to do this by looking at the 50 airports with the largest passenger turnover per year and compare them based on what restaurants are available and how restaurants were rated by passengers. The restaurant and rating data is supplied by foursquare, the airport location data is supplied by https://openflights.org/data.html and the data for the passenger traffic comes from Wikipedia (https://en.wikipedia.org/wiki/List_of_busiest_airports_by_passenger_traffic). 

Thus, we have three data sources:
    1. airport location data
    2. airport passenger traffic
    3. Data from the foursquare API to retrieve the different restaurants at each airport and their rating and category

### 5.2 Methodology

Firstly, the airport location data is merged with the airport passenger traffic data to generate a consistent data set of the top 50 airports and their respective locations.
Afterwards, this data is being visualized on a world map to identify which parts and cities of the world are represented and in what proportion.
The location of each airport, precisely the center of each airport, is now being used to retrieve data about restaurants in a radius of 10 kilometres, which is reasonable given the size of the respective airports. Here two different approaches are utilized:

    1. Retrieve all restaurants, their category and their ratings to measure the quality each airport has with regard to restaurants
    
    2. Retrieve popular restaurants and their category to measure the profile and coverage of diffenerent restaurant categories

In the first approach at first all restaurants in the 10km radius are being retrieved with the foursquare "search"-API. Afterwards this data is enriched with the ratings using the "venue"-API from foursquare. Only the restaurants with ratings are kept. 
After transforming the data an unsupervised machine learning technique is used to cluster the airports based on their similarity. Here the partitioning k-Means clustering algorithm is used. In order to answer the question at hand 5 clusters were chosen.

This algorithm was chosen because the idea is to find those airports that strongly differ from the majority, hence finding small clusters.
With respect to this approach, evaluating the quality of each airport, this must mean that airports in those "small" clusters perform either better than the rest or worse. Of course this holds only true under the assumption of a normal distribution respectively that the majority of airports perform average. This will be further evaluated in the results section.

Then, the data is being visualized to identify similar and distinct airport clusters.
Finally, those cluster are being examined on a high level by looking at which restaurant categories are rated best (and worst) to make recommendations for airport operators and restaurant owners what is working well at some airports but is not at others.

In the second approach, again restaurants in the 10km radius are being retrieved but this time with the foursquare "explore"-API which delivers the popular restaurant at each airport. Now this data is being transformed to identify which restaurant categories are highly represented at the respective airport and which are not. Then, the same partitioning clustering algorithm is used as in the first approach and the different clusters are being visualized on a map to find out similarities and differences between airports. Again, 5 clusters were chosen.

The partitioning cluster algorithm is chosen for similar reasons as in approach one. The difference here is that we want to find unique restaurant portfolios which are interesting to look at in comparison to the majority (small vs. large clusters).

Finally, those clusters are being examined on a high level by looking at the top 10 restaurant categories and how often they occur per cluster(results are weighted by amount of airports per cluster).

### 5.3 Results

Since two different approaches to answer the question of where an airport ranges when it comes to restaurants or where there might be potential at this airport to open up a new restaurant in a specific category also the results for those two approaches are different.

Unfortunately, the results for evaluating the quality of restaurants by looking at user ratings are biased since there is just not enough data, respectively for some airports only a few or none of the restaurants were rated. Moreover, not for all restaurant categories data was available. In addition, the "venue"-API only permits only 500 calls per day with a standard account and even then it is very slow which makes it pretty difficult to work with. Given that, even if more data were available, collecting only 10 ratings per airport would still not account for a comprehensive analysis.
Still, for 19 airports 326 rated venues were found and the clustering shows that 2 airports are quite unique (L.A. and JFK), there is one cluster with two airports and two bigger clusters. Especially, how the three small clusters compare to the bigger ones is of interest and can be find out by looking at the rating score and which restaurant categories were rated best.

Since approach number two only relied on the "explore"-API of foursquare which permits 95000 calls per day and data was readily available the results are much more useful and insightful. 
For the 50 airports 4645 restaurants or venues that contain a restaurant (such as hotels) in 379 different restaurant categories were used to cluster the airports. The assumption that many airports will be rather similar to each other and consequently are clustered together was right. Only 2 clusters comprise of 43 airports while the other 7 are split up in three different clusters.
This makes it particularly interesting to find out how they differentiate from the majority.

### 5.4 Discussion

The first part of the analysis tackles the restaurant quality. In order to make any recommendations for retaurants owners it is of particular interest to look at airports with average or bad ratings. 
This corresponds to a "red ocean strategy" where you try to push out existing competitors. This could be targeted at a single restaurant category with a weak rating e.g. Café's or American Restaurant's at Shanghai Pudong International Airport or Thai Restaurants at Suvarnabhumi Airport.It could also be targeted at an airport that in provides an average restaurant offering in general and it seems like airports such as Guangzhou Baiyun International Airport, Kuala Lumpur and Beijing Capital are worth looking at.

The second part of the analysis tackles the restaurant profile. In order to make any recommendations for retaurants owners it is of particular interest to look at airports with categories that are either not or only sparsely filled. This would correspond to a "blue ocean strategy" because it is about trying to occupy a spot with no one else has taken yet.
Especially in Cluster 0 but also in 1,2 and 3 it seems pretty unadvisable to open up a new coffee shop because there are plenty of them already while in Cluster 4 they are somewhat underrepresented compared to the others. Depending on where a restaurant owner or airport operator is located many more details could be derived from the analysis to have a starting point where new business opportunities are.

### 5.5 Conclusion

In this report airports where clustered based on the quality of their restaurants and based on the availability of restaurants in the different restaurant categories.Airport operators and restaurant owners alike can draw valuable insights from the provided analysis and get a starting point of where to look for new business opportunities.

Additionally, in line with the analysis a tool was developed that can easily serve to do the same analysis of the quality and/or profile of airports not just for restaurants but also for e.g. hotels, retail, IT or sports offerings by simply adjusting the query term of the location data provider foursquare. 

This analysis is highly dependent on the data quality of the location data provider. Therefore, it is advisable to incorporate other location data providers such as Google Places and merge the data to achieve a consistent analysis.